# Read from a Database - SELECT

> 🔗 You can see more detailed info at SQLModel's [official docs](https://sqlmodel.tiangolo.com/tutorial/select/)

We will be using the same "Heroes" example from the previous notebook. Make sure to delete the `database.db` file to start with a clean slate!

In [1]:
from typing import Optional

from sqlmodel import Field, Session, SQLModel, create_engine


class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    secret_name: str
    age: Optional[int] = None


sqlite_file_name = "database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"
engine = create_engine(sqlite_url, echo=True)


def create_db_and_tables():
    SQLModel.metadata.create_all(engine)


def create_heroes():
    hero_1 = Hero(name="Deadpond", secret_name="Dive Wilson")
    hero_2 = Hero(name="Spider-Boy", secret_name="Pedro Parqueador")
    hero_3 = Hero(name="Rusty-Man", secret_name="Tommy Sharp", age=48)

    with Session(engine) as session:
        session.add(hero_1)
        session.add(hero_2)
        session.add(hero_3)
        session.commit()


def main():
    create_db_and_tables()
    create_heroes()


In [2]:
# Run this cell to create the database, tables, and heroes
main()

2023-12-14 16:34:00,208 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-14 16:34:00,210 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hero")
2023-12-14 16:34:00,211 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-14 16:34:00,213 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("hero")
2023-12-14 16:34:00,213 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-14 16:34:00,215 INFO sqlalchemy.engine.Engine 
CREATE TABLE hero (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	secret_name VARCHAR NOT NULL, 
	age INTEGER, 
	PRIMARY KEY (id)
)


2023-12-14 16:34:00,216 INFO sqlalchemy.engine.Engine [no key 0.00060s] ()
2023-12-14 16:34:00,218 INFO sqlalchemy.engine.Engine COMMIT
2023-12-14 16:34:00,221 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-14 16:34:00,224 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age) VALUES (?, ?, ?) RETURNING id
2023-12-14 16:34:00,225 INFO sqlalchemy.engine.Engine [generated in 0.00017s (insertmanyvalues) 1/3 

## Read Data with SQL

```sql
SELECT id, name, secret_name, age
FROM hero
```

This says, more or less:

> Hey SQL database 👋, please go and SELECT some data for me.
>
> I'll first tell you the columns I want:
> 
> * id
> * name
> * secret_name
> * age
>
> And I want you to get them FROM the table called `"hero"`

Then it will return data that matches the query:

| id | name        | secret_name   | age |
|----|-------------|---------------|-----|
| 1  | Deadpond    | Dive Wilson   | null|
| 2  | Spider-Boy  | Pedro Parqueador | null|
| 3  | Rusty-Man   | Tommy Sharp   | 48  |

### Challenge

Give this a try in DB Browser! See if you can return data for _only_ the `id` and `name` columns.

### Good to know

Relational Databases store their data in tables and return the data in tables! However, you can query the database's _multiple_ tables and return it all back in a _single_ table!

## Read Data with Python and SQLModel

To do this, we will:

1. Create a Session to work with the DB
2. Create a `select` statement
3. Execute the statement
4. Work with the results

In [3]:
# Importing here to keep the relevant code together, but this would go in the imports section at the top of the file
from sqlmodel import select

# Define a function to select all heroes from the database. Maybe we could call it `get_all_heroes`?
def select_heroes():
    # 1. Any time we want to interact with the database, we need to create a session
    with Session(engine) as session:
        # 2. We define a statement to select all heroes
        statement = select(Hero)
        # 3. We execute the statement and store the results in a variable
        results = session.exec(statement)
        # 4. We can do anything with the results, like printing them one by one
        for hero in results:
            print(hero)

`statement = select(Hero)` is the same as this SQL:

```sql
select * from hero
```

Which means that it will return all columns and values from the `hero` table

In [4]:
# Call this function to see the results
select_heroes()

2023-12-14 16:34:08,212 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-14 16:34:08,216 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age 
FROM hero
2023-12-14 16:34:08,218 INFO sqlalchemy.engine.Engine [generated in 0.00139s] ()
secret_name='Dive Wilson' age=None id=1 name='Deadpond'
secret_name='Pedro Parqueador' age=None id=2 name='Spider-Boy'
secret_name='Tommy Sharp' age=48 id=3 name='Rusty-Man'
2023-12-14 16:34:08,220 INFO sqlalchemy.engine.Engine ROLLBACK


## Get Hereos in a List instead of an Iterable

Up to now we are using the results to iterate over them.

But for different reasons you might want to have the full list of Hero objects right away instead of just an iterable. For example, if you want to return them in a web API.

The special results object also has a method results.all() that returns a list with all the objects:

In [5]:
def select_heroes():
    with Session(engine) as session:
        statement = select(Hero)
        results = session.exec(statement)
        heroes = results.all()
        return heroes

In [6]:
select_heroes()

2023-12-14 16:34:12,384 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-14 16:34:12,386 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age 
FROM hero
2023-12-14 16:34:12,388 INFO sqlalchemy.engine.Engine [cached since 4.171s ago] ()
2023-12-14 16:34:12,389 INFO sqlalchemy.engine.Engine ROLLBACK


[Hero(secret_name='Dive Wilson', age=None, id=1, name='Deadpond'),
 Hero(secret_name='Pedro Parqueador', age=None, id=2, name='Spider-Boy'),
 Hero(secret_name='Tommy Sharp', age=48, id=3, name='Rusty-Man')]

We can also write this in a more compact form

In [7]:
def select_heroes():
    with Session(engine) as session:
        heroes = session.exec(select(Hero)).all()
        return heroes

select_heroes()

2023-12-14 16:34:14,117 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-14 16:34:14,120 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age 
FROM hero
2023-12-14 16:34:14,121 INFO sqlalchemy.engine.Engine [cached since 5.905s ago] ()
2023-12-14 16:34:14,122 INFO sqlalchemy.engine.Engine ROLLBACK


[Hero(secret_name='Dive Wilson', age=None, id=1, name='Deadpond'),
 Hero(secret_name='Pedro Parqueador', age=None, id=2, name='Spider-Boy'),
 Hero(secret_name='Tommy Sharp', age=48, id=3, name='Rusty-Man')]